In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [4]:
alcohol = "indicator alcohol consumption  20100830.xlsx"
gdp = "GDPpercapitaconstant2000US.xlsx"

data_alco = pd.read_excel(alcohol, header=0)
data_gdp = pd.read_excel(gdp, header=0)
data_gdp = data_gdp.set_index('Income per person (fixed 2000 US$)')
data_gdp.index.name = 'country'
data_alco.index.name = 'country'
countries = ['Portugal', 'Spain', 'United States', 'United Kingdom', 'Finland', 'Germany', 'Russia']
data_alco, data_gdp = data_alco.iloc[:, -10:], data_gdp.iloc[:, -10:]
data_alco, data_gdp = data_alco[data_alco.index.isin(countries)], data_gdp[data_gdp.index.isin(countries)]
data_gdp = data_gdp.loc[:, ('2005', '2008')]
data_alco = data_alco.loc[:, (2005, 2008)]
data_alco = data_alco.unstack().reset_index().rename({'level_0': 'year', 0: 'alco'}, axis=1)
data_gdp = data_gdp.unstack().reset_index().rename({'level_0': 'year', 0: 'gdp'}, axis=1)
data_alco['year'] = [str(i) for i in data_alco['year']]
data = pd.merge(data_alco, data_gdp, how='inner', on=('year', 'country'))

### Dashboard example

In [5]:
trace0 = go.Scatter(
    x=data['alco'][data['year']=='2008'].tolist(),
    y=data['gdp'][data['year']=='2008'].tolist(),
    text=list(data['country'][data['year']=='2008']),
    mode='markers',
    name='2008'
)
trace1 = go.Scatter(
    x=data['alco'][data['year']=='2005'].tolist(),
    y=data['gdp'][data['year']=='2005'].tolist(),
    text=list(data['country'][data['year']=='2005']),
    mode='markers',
    name='2005'
)
dt = [trace0, trace1]
updatemenus = list([
    dict(
        x = 0.1,
        xanchor = 'left',
        y = 1.0,
        yanchor = 'top'),
    dict(buttons=list(
        [
            dict(args = ["visible", [True, False]],
                 label = '2008',
                 method = 'restyle'),
            dict(args = ["visible", [False, True]],
                 label = '2005',
                 method = 'restyle'),
            dict(args = ["visible", [True, True]],
                 label = 'All',
                 method = 'restyle')
        ]),
    )
])


layout = dict(
            title = 'Alcohol consumption & gdp', 
            updatemenus=updatemenus, margin = dict( t=30, b=30, l=0, r=0 )
            )

fig = dict(data=dt, layout=layout)
iplot(fig)


In [6]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')